In [1]:
import pandas as pd
import numpy as np
import seaborn as sns


df = pd.read_csv("emotion_text_data.csv")
df.head()

,ID,Emotion,Text
0,399172782,angry,"Ел жаңалықтарына келсек, Өскемен, Алматы, Қара..."
1,399172782,angry,Олар үйіржұлдыздың тығыздығы жоғары өзек өңірі...
2,399172782,angry,"Оқуымды жақсы бағамен бітіріп, өзімнің туып өс..."
3,399172782,angry,"Егер карта бетіне қарап, геофизикалық өлшеммен..."
4,399172782,angry,Жылдық орташа табыстарыңыз қанша?


In [5]:
df['Emotion'].value_counts()

Emotion
neutral     1674
sad         1648
angry       1637
fear        1612
happy       1610
surprise    1605
Name: count, dtype: int64

In [7]:
import neattext.functions as nfx

# Remove the user handles
df['Clean_Text'] = df['Text'].apply(nfx.remove_userhandles)

In [8]:
# Remove the stopwords
df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_stopwords)

In [11]:
kazakh_stopwords = [
    'мен', 'сен', 'ол', 'біз', 'сіз', 'бұл', 'анау', 'және', 'бірақ', 'немесе', 'осы', 'қалай', 'қандай',
    'ма', 'ме', 'па', 'пе', 'е', 'де', 'да', 'та', 'те', 'тағы', 'бол', 'бар', 'жоқ', 'иә', 'жоқ', 'барлық',
    'бәрі', 'содан', 'мұнда', 'сонда', 'сондықтан', 'өйткені', 'сондай', 'мұндай', 'әлде', 'қайда', 'неге',
    'кім', 'не', 'қанша', 'неше', 'қашан', 'сол', 'мына', 'мысалы', 'менің', 'сенің', 'оның', 'біздің', 'сіздің',
    'олардың', 'қасында', 'үстінде', 'астында', 'ішінде', 'сыртында', 'бұл жерде', 'сол жерде', 'әлі', 'барлық',
    'ешқандай', 'кейбір', 'осылар', 'солар', 'біреу', 'ешкім', 'әлдеқашан', 'кейде', 'жиі', 'сирек', 'бұған', 
    'содан кейін', 'бірнеше', 'шамамен', 'яғни', 'қазір', 'мүмкін', 'әрине', 'тағы да', 'енді', 'мұнымен',
    'соншалықты', 'тіпті', 'бұрын', 'кейін', 'әр', 'соншама', 'бұлар', 'мұншама', 'бірақ та', 'осында'
]


In [12]:
def remove_kazakh_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in kazakh_stopwords]
    return ' '.join(filtered_words)

In [13]:
df['Clean_Text'] = df['Clean_Text'].apply(remove_kazakh_stopwords)

In [14]:
# Remove the stopwords
df.head()

,ID,Emotion,Text,Clean_Text
0,399172782,angry,"Ел жаңалықтарына келсек, Өскемен, Алматы, Қара...","Ел жаңалықтарына келсек, Өскемен, Алматы, Қара..."
1,399172782,angry,Олар үйіржұлдыздың тығыздығы жоғары өзек өңірі...,Олар үйіржұлдыздың тығыздығы жоғары өзек өңірі...
2,399172782,angry,"Оқуымды жақсы бағамен бітіріп, өзімнің туып өс...","Оқуымды жақсы бағамен бітіріп, өзімнің туып өс..."
3,399172782,angry,"Егер карта бетіне қарап, геофизикалық өлшеммен...","Егер карта бетіне қарап, геофизикалық өлшеммен..."
4,399172782,angry,Жылдық орташа табыстарыңыз қанша?,Жылдық орташа табыстарыңыз қанша?


In [15]:
x = df['Clean_Text']
y = df['Emotion']

In [16]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [22]:
pipe_lr = Pipeline(steps=[('cv', CountVectorizer()), ('lr', LogisticRegression(class_weight='balanced'))])

pipe_lr.fit(x_train,y_train)
pipe_lr.score(x_test,y_test)



0.18290190735694822

In [24]:
pipe_lr = Pipeline(steps=[('cv', CountVectorizer(lowercase=True, stop_words=kazakh_stopwords, max_features=10000)),
                          ('lr', LogisticRegression())])

In [25]:
pipe_lr = Pipeline(steps=[('cv', CountVectorizer()),
                          ('lr', LogisticRegression(C=0.1, solver='liblinear'))])

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

pipe_lr = Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=10000)),
                          ('lr', LogisticRegression())])

In [27]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe_lr, x_train, y_train, cv=5)

array([0.16350365, 0.17518248, 0.14671533, 0.1620438 , 0.16934307])

In [28]:
from sklearn.naive_bayes import MultinomialNB

pipe_nb = Pipeline(steps=[('tfidf', TfidfVectorizer()), ('nb', MultinomialNB())])
pipe_nb.fit(x_train, y_train)
pipe_nb.score(x_test, y_test)

0.18119891008174388

In [29]:
print(y_train.value_counts())

Emotion
neutral     1183
sad         1166
surprise    1140
fear        1138
angry       1117
happy       1106
Name: count, dtype: int64


In [30]:
pipe_lr = Pipeline(steps=[('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
                          ('lr', LogisticRegression())])
